Hier werden alle Daten aus verschiedenen Quellen zu einem großen Dataframe zusammen gefasst. Das sind zum einen die Daten welche uns vom Kurs zu Verfügung gestellt wurden, als auch die daten welche wir selber noch gefunden haben.

In [1]:
#packete für data handling laden
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [7]:
#Einlesen der Basisidaten, welche wir zur verfügung gestellt bekommen haben
#Wetterdaten
wetter = pd.read_csv("../wetter.csv")
print(wetter.head())
print(wetter.tail())
print(wetter.shape)

#Umsatzdaten
umsatz = pd.read_csv("../umsatzdaten_gekuerzt.csv")
print(umsatz.head())
print(umsatz.tail())
print(umsatz.shape)

#Kieler Woche Daten
kiwo = pd.read_csv("../kiwo.csv")
print(kiwo.head())
print(kiwo.tail())
print(kiwo.shape)

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
0  2012-01-01         8.0      9.8250                   14        58.0
1  2012-01-02         7.0      7.4375                   12         NaN
2  2012-01-03         8.0      5.5375                   18        63.0
3  2012-01-04         4.0      5.6875                   19        80.0
4  2012-01-05         6.0      5.3000                   23        80.0
           Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode
2596  2019-07-28         3.0     23.3500                   14         5.0
2597  2019-07-29         6.0     25.2500                    7        61.0
2598  2019-07-30         7.0     20.7375                    8        61.0
2599  2019-07-31         6.0     20.4500                    7        61.0
2600  2019-08-01         5.0     21.0625                    9        61.0
(2601, 5)
        Datum  Warengruppe      Umsatz
0  2013-07-01            1  148.828353
1  2013-07-02            1  159.793757
2  

Die Umsatzdaten sind im long format nach Warengruppen. Bedeutet, dass jedes Datum mehrmals vorkommt. Das ist zum mergen der Daten nicht ideal. Daher den dataframe vom long format ins wide format bringen. 
In diesem Zuge werden direkt auch die Warengruppen richtig benannt

In [8]:
#get the umsart data in the wide format
# Pivot the DataFrame
umsatz_shaped = umsatz.pivot(index='Datum', columns='Warengruppe', values='Umsatz')

# Reset the index to make 'date' a column again
umsatz_shaped.reset_index(inplace=True)

#get colums names right
umsatz_shaped.columns = ['Datum', 'Brot', 'Broetchen', 'Crossaint', 'Konditorei', 'Kuchen', 'Saisonbrot']

print(umsatz_shaped.head())
print(umsatz_shaped.tail())    
print(umsatz_shaped.shape)

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
           Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
1814  2018-07-27  198.225523  677.415844  333.774099   65.563734  297.867853   
1815  2018-07-28  214.470393  716.576825  371.819770   54.047422  290.024007   
1816  2018-07-29  129.801991  721.822447  433.820697  128.952852  335.823557   
1817  2018-07-30  148.541340  616.358562  348.770346   71.822977  324.965348   
1818  2018-07-31  123.075682  586.081666 

In [9]:
#aufräumen der Daten. NaN durch 0 ersetzen
umsatz_shaped.fillna(0, inplace=True)

#Eine Spalte hinzufügen in welcher der Gesamtumsatz steht
umsatz_shaped['Umsatz_total'] = umsatz_shaped['Brot'] + umsatz_shaped['Broetchen'] + umsatz_shaped['Crossaint'] + umsatz_shaped['Konditorei'] + umsatz_shaped['Kuchen'] + umsatz_shaped['Saisonbrot']
umsatz_shaped['Umsatz_total'] = umsatz_shaped['Umsatz_total'].astype(int)
print(umsatz_shaped.head())

        Datum        Brot   Broetchen   Crossaint  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  
0         0.0          1269  
1         0.0          1430  
2         0.0          1124  
3         0.0          1184  
4         0.0          1288  


In [18]:
#nun können die Umsatzdaten mit den Wetterdaten zusammengeführt werden
#merge the data
data = pd.merge(wetter, umsatz_shaped, how='left', on='Datum')
print(data.head())
print(data.shape)

#es werden nun erstmal alle Zeilen behalten auch wenn es keine Umsatzdaten gibt

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  Brot  \
0  2012-01-01         8.0      9.8250                   14        58.0   NaN   
1  2012-01-02         7.0      7.4375                   12         NaN   NaN   
2  2012-01-03         8.0      5.5375                   18        63.0   NaN   
3  2012-01-04         4.0      5.6875                   19        80.0   NaN   
4  2012-01-05         6.0      5.3000                   23        80.0   NaN   

   Broetchen  Crossaint  Konditorei  Kuchen  Saisonbrot  Umsatz_total  
0        NaN        NaN         NaN     NaN         NaN           NaN  
1        NaN        NaN         NaN     NaN         NaN           NaN  
2        NaN        NaN         NaN     NaN         NaN           NaN  
3        NaN        NaN         NaN     NaN         NaN           NaN  
4        NaN        NaN         NaN     NaN         NaN           NaN  
(2601, 12)


In [ ]:
# und nun noch die Kieler Woche Daten hinzufügen
data = pd.merge(data, kiwo, how='left', on='Datum')
print(data.head())
print(data.shape)

#im moment steht da eine 1.0 wenn es Kieler Woche ist und eine NaN wenn nicht. Das ändern wir jetzt
data['KielerWoche'] = data['KielerWoche'].fillna(0)
data['KielerWoche'] = data['KielerWoche'].astype(int)
print(data.head())	

        Datum  Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  Brot  \
0  2012-01-01         8.0      9.8250                   14        58.0   NaN   
1  2012-01-02         7.0      7.4375                   12         NaN   NaN   
2  2012-01-03         8.0      5.5375                   18        63.0   NaN   
3  2012-01-04         4.0      5.6875                   19        80.0   NaN   
4  2012-01-05         6.0      5.3000                   23        80.0   NaN   

   Broetchen  Crossaint  Konditorei  Kuchen  Saisonbrot  Umsatz_total  \
0        NaN        NaN         NaN     NaN         NaN           NaN   
1        NaN        NaN         NaN     NaN         NaN           NaN   
2        NaN        NaN         NaN     NaN         NaN           NaN   
3        NaN        NaN         NaN     NaN         NaN           NaN   
4        NaN        NaN         NaN     NaN         NaN           NaN   

   KielerWoche  
0          NaN  
1          NaN  
2          NaN  
3          N

Nun haben wir alle Basis